In [1]:
from dotenv import dotenv_values
config = dotenv_values("../.env")

import os
import pandas as pd
from datetime import date, datetime, timedelta
import time

from riotwatcher import LolWatcher, ApiError

""" 
API Rate Limits:
20 requests every 1 seconds(s)
100 requests every 2 minutes(s)
"""

""" initialize API """
watcher = LolWatcher(config['riotwatcher_api_key'])


""" user-defined variables """
list_summoner_name = ['AdAstraData', 'Storm and Sin']
var_summoner_region = 'euw1'

In [27]:
from match_info.info_from_match import push_sql_tbl_summoner_matches

import pandas as pd 
import time

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL

engine = create_engine("mssql+pyodbc://DESKTOP-CBU1M8B/DataScience?driver=ODBC+Driver+17+for+SQL+Server")

""" for each summoner_name """
for var_summoner_name in list_summoner_name:

    """ API GET puuid """
    var_summoner_puuid = watcher.summoner.by_name(region=var_summoner_region, summoner_name=var_summoner_name)['puuid']

    """ fetch match_list for each week in the last n weeks (num_weeks) """

    list_all_matches = []

    num_weeks = 1

    ### define current_week
    var_current_date = date.today()
    var_current_year, var_current_week, var_current_weekday = var_current_date.isocalendar()

    for week_num in range(num_weeks):
        
        # define iterator
        if var_current_week > week_num:
            var_week_iterator = var_current_week - week_num
            var_year_iterator = var_current_year
        else:
            var_week_iterator = 52 - (week_num - var_current_week)
            var_year_iterator = var_current_year - 1
        
        # define startdate_iterator (Mon)
        var_startdate_iterator = datetime.fromisocalendar(var_year_iterator, var_week_iterator, 1)
        
        # define enddate_iterator (Sun)
        var_enddate_iterator = datetime.fromisocalendar(var_year_iterator, var_week_iterator, 7)

        # print(var_startdate_iterator, var_enddate_iterator)

        """ API GET week_matches played by puuid """
        list_week_matches = watcher.match.matchlist_by_puuid(
            region=var_summoner_region, 
            puuid=var_summoner_puuid,
            start_time=int(var_startdate_iterator.timestamp()), ### epoch timestamp in seconds
            end_time=int(var_enddate_iterator.timestamp()), ### epoch timestamp in seconds
            count=100, #count=100,
        )

        list_all_matches.extend(list_week_matches)

        time.sleep(2)

    """ API GET match_info for each match_id of all those found above """

    # print(list_all_matches)

    """ for each match_id """
    for var_match_id in list_all_matches:
        
        print(var_summoner_name, var_match_id) ###
            
        integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

        push_sql_tbl_summoner_matches(
            watcher, 
            var_summoner_puuid, 
            var_summoner_name, 
            var_summoner_region, 
            var_match_id, 
            engine, 
            integration_datetime
        )

        time.sleep(2)


In [28]:
from match_timeline.events_from_match_timeline import get_events_from_match_timeline

from match_timeline.champion_level_up import push_sql_tbl_events_champion_level_up
from match_timeline.skill_level_up import push_sql_tbl_events_skill_level_up

from match_timeline.ward_placed import push_sql_tbl_events_ward_placed
from match_timeline.ward_kill import push_sql_tbl_events_ward_kill

from match_timeline.champion_kill import push_sql_tbl_events_champion_kill

import pandas as pd 
import time

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL

engine = create_engine("mssql+pyodbc://DESKTOP-CBU1M8B/DataScience?driver=ODBC+Driver+17+for+SQL+Server")

""" fetch all matches from SQL tbl [DataScience].[dbo].[tbl_summoner_matches] """

list_tbl_summoner_matches_match_id = pd.read_sql_query("""
SELECT DISTINCT match_id
FROM tbl_summoner_matches
""", con=engine)['match_id'].tolist()

""" for each match_id """
for var_match_id in list_tbl_summoner_matches:

    watcher_match_timeline = watcher.match.timeline_by_match(region=var_summoner_region,match_id=var_match_id)

    (
        events_all,
        events_pause_end,
        events_ward_placed,
        events_ward_kill,
        events_champion_level_up,
        events_skill_level_up,
        events_item_purchases,
        events_item_undo,
        events_item_destroyed,
        events_item_sold,
        events_champion_kill,
        events_champion_special_kill,
        events_elite_monster_kill,
        events_turret_plate_destroyed,
        events_building_kill,
    ) = get_events_from_match_timeline(watcher_match_timeline)

    integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

    df_puuid = pd.json_normalize(watcher_match_timeline['info']['participants'])
    dict_puuid = dict(df_puuid.values)

    if events_ward_placed.empty:
        pass
    else:
        push_sql_tbl_events_ward_placed(var_match_id, events_ward_placed, dict_puuid, engine, integration_datetime)

    if events_ward_kill.empty:
        pass
    else:
        push_sql_tbl_events_ward_kill(var_match_id, events_ward_kill, dict_puuid, engine, integration_datetime)

    if events_champion_level_up.empty:
        pass
    else:
        push_sql_tbl_events_champion_level_up(var_match_id, events_champion_level_up, dict_puuid, engine, integration_datetime)

    if events_skill_level_up.empty:
        pass
    else:
        push_sql_tbl_events_skill_level_up(var_match_id, events_skill_level_up, dict_puuid, engine, integration_datetime)

    if events_champion_kill.empty:
        pass
    else:
        push_sql_tbl_events_champion_kill(var_match_id, events_champion_kill, dict_puuid, engine, integration_datetime)



In [29]:
###

In [9]:
from match_info.info_from_match import push_sql_tbl_summoner_matches

import pandas as pd 
import time

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL

engine = create_engine("mssql+pyodbc://DESKTOP-CBU1M8B/DataScience?driver=ODBC+Driver+17+for+SQL+Server")

""" fetch all matches from SQL tbl [DataScience].[dbo].[tbl_events_champion_level_up] """

list_tbl_events_champion_level_up_puuid_id = pd.read_sql_query("""
SELECT DISTINCT puuid_id
FROM tbl_events_champion_level_up
WHERE puuid_id IS NOT NULL
""", con=engine)['puuid_id'].tolist()

""" for each puuid_id """
for var_summoner_puuid in list_tbl_events_champion_level_up_puuid_id:

    """ API GET puuid """
    var_summoner_name = watcher.summoner.by_puuid(region=var_summoner_region, encrypted_puuid=var_summoner_puuid)['name']

    print(var_summoner_name, var_summoner_puuid)

    """ fetch match_list for each week in the last n weeks (num_weeks) """

    list_all_matches = []

    num_weeks = 10

    ### define current_week
    var_current_date = date.today()
    var_current_year, var_current_week, var_current_weekday = var_current_date.isocalendar()

    for week_num in range(num_weeks):
        
        # define iterator
        if var_current_week > week_num:
            var_week_iterator = var_current_week - week_num
            var_year_iterator = var_current_year
        else:
            var_week_iterator = 52 - (week_num - var_current_week)
            var_year_iterator = var_current_year - 1
        
        # define startdate_iterator (Mon)
        var_startdate_iterator = datetime.fromisocalendar(var_year_iterator, var_week_iterator, 1)
        
        # define enddate_iterator (Sun)
        var_enddate_iterator = datetime.fromisocalendar(var_year_iterator, var_week_iterator, 7)

        # print(var_startdate_iterator, var_enddate_iterator)

        """ API GET week_matches played by puuid """
        list_week_matches = watcher.match.matchlist_by_puuid(
            region=var_summoner_region, 
            puuid=var_summoner_puuid,
            start_time=int(var_startdate_iterator.timestamp()), ### epoch timestamp in seconds
            end_time=int(var_enddate_iterator.timestamp()), ### epoch timestamp in seconds
            count=100, #count=100,
        )

        list_all_matches.extend(list_week_matches)

        time.sleep(2)

    """ API GET match_info for each match_id of all those found above """

    # print(list_all_matches)

    """ for each match_id """
    for var_match_id in list_all_matches:
        
        print(var_summoner_name, var_match_id) ###

        integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

        push_sql_tbl_summoner_matches(
            watcher, 
            var_summoner_puuid, 
            var_summoner_name, 
            var_summoner_region, 
            var_match_id, 
            engine, 
            integration_datetime
        )

        time.sleep(2)


YourMumWantsMe --PbzqFmXFEvfGlgIr0c6PB9DdY2vQjG7Sgttpe8_UekSMm3ejYQ90yBnK_qg14XC9GDRNjUxR6kgw
YourMumWantsMe EUW1_6148919916
YourMumWantsMe EUW1_6148323708
YourMumWantsMe EUW1_6147505121
YourMumWantsMe EUW1_6147471539
YourMumWantsMe EUW1_6147422821
YourMumWantsMe EUW1_6147373616
YourMumWantsMe EUW1_6145626832
YourMumWantsMe EUW1_6145595108
YourMumWantsMe EUW1_6145558044
YourMumWantsMe EUW1_6145527078
YourMumWantsMe EUW1_6145514549
YourMumWantsMe EUW1_6145481761
YourMumWantsMe EUW1_6145454052
YourMumWantsMe EUW1_6145427312
YourMumWantsMe EUW1_6145395507
YourMumWantsMe EUW1_6145366662
YourMumWantsMe EUW1_6145337212
YourMumWantsMe EUW1_6145313258
YourMumWantsMe EUW1_6135612571
YourMumWantsMe EUW1_6135594995
YourMumWantsMe EUW1_6135572328
YourMumWantsMe EUW1_6135551909
YourMumWantsMe EUW1_6135527339
YourMumWantsMe EUW1_6134561721
YourMumWantsMe EUW1_6134519361
YourMumWantsMe EUW1_6134486606
YourMumWantsMe EUW1_6134457377
YourMumWantsMe EUW1_6134334777
YourMumWantsMe EUW1_6127635316
YourMum

In [30]:
###

In [22]:
watcher_match_timeline

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6166464245',
  'participants': ['pzSt9aY2l5uIGLvcSxHye1ni88P2d6KLlcgBIC7FMHRHFnhjJB5ziPpxdKC3ZhOvfbMV7JE7Kssvcw',
   'kl-OG7WjpxgC8nzkBu54dnRQm2BNko1XAm0Id3tJ_eeeJ6AVAekbrWeVxe-o27r5rNtCJCv8-bwITg',
   'vof2EdjHrAFKyBokAn8HTLzRD6OJ8mJvQBybjGcxfDF0LNRSmVev1qHtW6t7iI5ajI2pZSIs8LHh2A',
   'sZ76RomH07SWUrj_VuyIS8EkdtVwNSIcmKkmJKy9aSEGDZpnhBVLSqFSZioeHz7K5H0VLj41zPaCGw',
   'SL0TEoksIwLmcRBGer0BpNEGtoO8CXZuYfjBiR24O3L7tOy5mP8tdPrKsKsznzZQ3Kiumdbf5s6kMQ',
   '3CNwFMMgPIZ_H6FQxwzdwnIcqruKTG0EczKogxgWw57sxOejmjMnT97NHXDmf4LhV6UArBfo6JYuAw',
   'u8bTReNaxa-2JKMCg2ShusJN2CLPpJHmb7GKQxKx77Jaz-CV2IFED4bRaytDp6vm31Kd-ahkTTgeiQ',
   'iMo7fomjD1qMYfqOA3d2TZkJ1E181_EPFEdnEQhlEr1yS__Idj8-doxosG-sa9Lstw91vwIyY-Weng',
   'M1jvBNS2vOckcErCYl3UnWjSkmoVXZXpFapHgHLvw1SIvzJk_OG9fCR_ov0rbxzAKeruDUkvt8tYoA',
   'pdmWhZjkPNzJ2FIcmd5YfkxyNmFTCVbMbL2WOaWgMdck2zc2QqNXaLRon8uEYrx30WwVO7uZ8lt0Gg']},
 'info': {'frameInterval': 60000,
  'frames': [{'events': [{'realT

In [19]:
""" for each match_id """
for var_match_id in ['EUW1_6122482710']: ### in list_tbl_summoner_matches:

    print(var_match_id)

    watcher_match_timeline = watcher.match.timeline_by_match(region=var_summoner_region,match_id=var_match_id)

    (
        events_all,
        events_pause_end,
        events_ward_placed,
        events_ward_kill,
        events_champion_level_up,
        events_skill_level_up,
        events_item_purchases,
        events_item_undo,
        events_item_destroyed,
        events_item_sold,
        events_champion_kill,
        events_champion_special_kill,
        events_elite_monster_kill,
        events_turret_plate_destroyed,
        events_building_kill,
    ) = get_events_from_match_timeline(watcher_match_timeline)

    integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

    df_puuid = pd.json_normalize(watcher_match_timeline['info']['participants'])
    dict_puuid = dict(df_puuid.values)

import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL

engine = create_engine("mssql+pyodbc://DESKTOP-CBU1M8B/DataScience?driver=ODBC+Driver+17+for+SQL+Server")

dt = events_ward_placed

events_all.type.value_counts()
# dt.dropna(axis=1, inplace=True)

# dt['match_id'] = match_id
# dt['puuid_id'] = dt['creatorId'].map(dict_puuid)
# dt['event_id'] = dt['type'].str.lower()
# dt['event_timestamp'] = dt['timestamp']
# dt['ward_type'] = dt['wardType'].str.lower()
# dt['integration_datetime'] = integration_datetime

# dt = dt[['integration_datetime', 'match_id', 'puuid_id', 'event_id', 'event_timestamp', 'ward_type']]

# df_tmp = pd.read_sql_query("SELECT * from tbl_events_ward_placed WHERE match_id = '%s'" %(match_id), con=engine)

# if df_tmp.empty:
#     dt.to_sql('tbl_events_ward_placed', con=engine, if_exists='append', index=False)
# else:
#     pass

EUW1_6122482710


SKILL_LEVEL_UP           142
ITEM_PURCHASED           142
LEVEL_UP                 133
ITEM_DESTROYED            94
CHAMPION_KILL             75
CHAMPION_SPECIAL_KILL     15
BUILDING_KILL              6
ITEM_UNDO                  5
ITEM_SOLD                  2
PAUSE_END                  1
GAME_END                   1
Name: type, dtype: int64

In [ ]:

    # """ API GET puuid """
    # var_summoner_puuid = watcher.summoner.by_name(region=var_summoner_region, summoner_name=var_summoner_name)['puuid']


In [ ]:

# """ fetch item_list from data_dragon and create dict with item_id : item_name """
# data_dragon_versions = watcher.data_dragon.versions_for_region(region=var_region)
# data_dragon_item_version = data_dragon_versions['n']['item']

# item_list = watcher.data_dragon.items(data_dragon_item_version)

# dict_item_id_item_name = {}

# for item_id in item_list['data'].keys():
#     df_item_id = pd.json_normalize(data=item_list['data'][item_id])
#     item_name = df_item_id['name'].to_string(index=False)
#     dict_item_id_item_name[item_id] = item_name


In [12]:

watcher_match['info'].keys()
# dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 
# 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 
# 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])


# """ global information per participant/ summoner """
watcher_match['info']['participants'][2]

# """ global information per team """
# watcher_match['info']['teams'][0]


NameError: name 'watcher_match' is not defined

In [ ]:
# watcher_match_timeline.keys()
# # dict_keys(['metadata', 'info'])

# watcher_match_timeline['metadata'].keys()
# # dict_keys(['dataVersion', 'matchId', 'participants'])

# watcher_match_timeline['info'].keys()
# # dict_keys(['frameInterval', 'frames', 'gameId', 'participants'])

# """ game_participants """
# watcher_match_timeline['info']['participants']


In [ ]:

""" 
Champion Kill
"""


def push_sql_tbl_events_champion_kill(engine, events_champion_kill, dict_puuid, match_id = watcher_match_timeline['metadata']['matchId']):


push_sql_tbl_events_champion_kill(engine, events_champion_kill, dict_puuid)


In [ ]:

# dtt = pd.json_normalize(events_champion_kill['victimDamageReceived'])

# # for this match_id

# # for each champion kill:
# for i in range(dtt.shape[0]):

# # for each contribution to champion kill:
#     for j in range(len(dtt.iloc[i])):

#         if dtt.iloc[i, j] == 'None':
#             pass
#         else:
#             pass
#             # print(dtt.iloc[i, j])
 
#         print(dtt.iloc[i, j])

In [ ]:


events_champion_special_kill

events_elite_monster_kill

events_turret_plate_destroyed

events_building_kill


In [ ]:

# for var_match_id in df_match_info['match_id'].unique():
#     df_tmp = pd.read_sql_query("SELECT * from tbl_match_info WHERE match_id = '%s'" %(var_match_id), con=engine)
    
#     if df_tmp.empty:
#         df_append = df_match_info.query("match_id == @var_match_id")
#         df_append.to_sql('tbl_match_info', con=engine, if_exists='append', index=False)
#     else:
#         pass

In [ ]:
### 1 FRAME PER MIN
datetime.fromtimestamp(watcher_match_timeline['info']['frames'][1]['timestamp'] / 1000).strftime('%H:%M:%S')
datetime.fromtimestamp(watcher_match_timeline['info']['frames'][2]['timestamp'] / 1000).strftime('%H:%M:%S')
datetime.fromtimestamp(watcher_match_timeline['info']['frames'][3]['timestamp'] / 1000).strftime('%H:%M:%S')

### INSIDE EACH FRAME, per participant
watcher_match_timeline['info']['frames'][1]['participantFrames']['1'].keys()
# dict_keys(['championStats', 'currentGold', 'damageStats', 'goldPerSecond', 
# 'jungleMinionsKilled', 'level', 'minionsKilled', 'participantId', 'position', 
# 'timeEnemySpentControlled', 'totalGold', 'xp'])

watcher_match_timeline['info']['frames'][20]['participantFrames']['1']['totalGold']

watcher_match_timeline['info']['frames'][20]['participantFrames']['1']['xp']


10209

In [ ]:
dict_match_info = {
    "match_id": [],
    "team_id": [],
    "participant_puuid": [],
    "summoner_name": [],
    "gold_earned": [],
}

""" for each match_id """
for var_match_id in last_matches_puuid[0:100]:

    """ GET summoner puuids """
    watcher_match = watcher.match.by_id(region=var_region, match_id=var_match_id)

    for puuid_num in range(len(watcher_match['metadata']['participants'])):
        puuid_features = watcher_match['info']['participants'][puuid_num]

        dict_match_info['match_id'].append(var_match_id)
        dict_match_info['team_id'].append(puuid_features['teamId'])
        dict_match_info['participant_puuid'].append(puuid_features['puuid'])
        dict_match_info['summoner_name'].append(puuid_features['summonerName'])
        dict_match_info['gold_earned'].append(puuid_features['goldEarned'])
    
    """ time delay with time.sleep """
    time.sleep(1)


In [ ]:
df_match_info = pd.DataFrame.from_dict(dict_match_info)
df_match_info['team_id'] = ['Blue' if team_id == 100 else 'Red' for team_id in df_match_info['team_id']]
df_match_info

list_all_match_participants = df_match_info['summoner_name'].unique()
len(list_all_match_participants)

853

In [ ]:
len(list_all_match_participants)

853

In [ ]:
list_all_match_participants[25:26]

array(['Alecostry22'], dtype=object)

In [ ]:
""" user-defined variables """
var_region = "euw1"
# var_summoner_name= "AdAstraData"

dict_match_info = {
    "match_id": [],
    "team_id": [],
    "participant_puuid": [],
    "summoner_name": [],
    "gold_earned": [],
}

for var_participant_name in list_all_match_participants[27:30]:
    
    """ GET participant puuid """
    summoner_puuid = watcher.summoner.by_name(region=var_region, summoner_name=var_participant_name)
    var_participant_puuid = summoner_puuid['puuid']
    
    """ GET last 100 matches played by puuid """
    last_matches_puuid = watcher.match.matchlist_by_puuid(
        region=var_region, 
        puuid=var_participant_puuid, 
        count=100
    )

    """ time delay with time.sleep """
    time.sleep(1)

    """ for each match_id """
    for var_match_id in last_matches_puuid[0:100]:
        
        """ GET summoner puuids """
        watcher_match = watcher.match.by_id(region=var_region, match_id=var_match_id)
        
        for puuid_num in range(len(watcher_match['metadata']['participants'])):
            puuid_features = watcher_match['info']['participants'][puuid_num]
            
            dict_match_info['match_id'].append(var_match_id)
            dict_match_info['team_id'].append(puuid_features['teamId'])
            dict_match_info['participant_puuid'].append(puuid_features['puuid'])
            dict_match_info['summoner_name'].append(puuid_features['summonerName'])
            dict_match_info['gold_earned'].append(puuid_features['goldEarned'])

        """ time delay with time.sleep """
        time.sleep(1)


# EUW1_5021326873


In [ ]:
df_match_info = pd.DataFrame.from_dict(dict_match_info)
df_match_info['team_id'] = ['Blue' if team_id == 100 else 'Red' for team_id in df_match_info['team_id']]
df_match_info

,match_id,team_id,participant_puuid,summoner_name,gold_earned
0,EUW1_6161945478,Blue,A2zEx9varFrcDbkKlqdGuD0dr5dCO0pcB4MRqH6N6Qha6J...,poutmasmurf,6898
1,EUW1_6161945478,Blue,MEVanfuJg9PZqtZbaJPB2JbkU9gbb83q9wDJobu06PXpnf...,FriiZoK7,9587
2,EUW1_6161945478,Blue,h3C486P2TXijAzf80wqo8wuscyhoo1Qypzd1pvu0JEYb6a...,haus300,8852
3,EUW1_6161945478,Blue,me53WpNeg3rqK2j0xb30wxMWFQ37MMZdF1RzPvSi38SR1j...,godbutbackwards,6209
4,EUW1_6161945478,Blue,Jl4DjLQgMwQK-9xga9-ddMeEgsq5NGUFy_Gy9O-SQkHQhW...,davidjimi,5610
...,...,...,...,...,...
1926,EUW1_5162267977,Blue,-C2u2z2GvgyjMrwuVLWcrkVN3joW392KZSMWtLRCHAO5v_...,popcgae,4749
1927,EUW1_5162267977,Blue,n9xTZEVS8IP4C0arauuyyHyrI_NvR0bz7D-uQV5rJdY9Nc...,Kanriee,7994
1928,EUW1_5162267977,Blue,joR0O9lnLuhRuU8z0OfKJxJU9xxOEguG-HCI_Yarrjm-Fp...,SyRuB1,14513
1929,EUW1_5162267977,Blue,GQIS690djRh5tNELDpJNdxcvyVlwGcqShzRI3krMOeJbjj...,skeith241,6349


In [ ]:
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite:///riotwatcher_api_dt_match.db', echo=False)

# df_append = pd.DataFrame(
#     [['test', 'test', 'test', 'test', 'test']],
#     columns = ['match_id', 'team_id', 'participant_puuid', 'summoner_name', 'gold_earned']
# )

# df_append.to_sql('tbl_match_info', con=engine, if_exists='append', index=False)

for var_match_id in df_match_info['match_id'].unique():
    df_tmp = pd.read_sql_query("SELECT * from tbl_match_info WHERE match_id = '%s'" %(var_match_id), con=engine)
    
    if df_tmp.empty:
        df_append = df_match_info.query("match_id == @var_match_id")
        df_append.to_sql('tbl_match_info', con=engine, if_exists='append', index=False)
    else:
        pass


In [ ]:
df = pd.read_sql_query("SELECT DISTINCT * from tbl_match_info", con=engine)
df

,match_id,team_id,participant_puuid,summoner_name,gold_earned
0,test,test,test,test,test
1,EUW1_6162024635,Blue,pdmWhZjkPNzJ2FIcmd5YfkxyNmFTCVbMbL2WOaWgMdck2z...,Storm and Sin,6124
2,EUW1_6162024635,Blue,bAbkdqqlpnuDlKFo32I_PBfi-yO2R8eBcD4qo9hqkAZ8Nm...,Ariiial,4539
3,EUW1_6162024635,Blue,bZZNu22YOLpn8iDq1FCmW2u4fjW6hNGed3YKTh9LvQeNI6...,AdAstraData,6727
4,EUW1_6162024635,Blue,9fZyZmC705BV1nflyxYDE_qtc4EnxO0SsCjIZ6DG2PV-Fi...,Glockenhorst,9177
...,...,...,...,...,...
20413,EUW1_5162267977,Blue,-C2u2z2GvgyjMrwuVLWcrkVN3joW392KZSMWtLRCHAO5v_...,popcgae,4749
20414,EUW1_5162267977,Blue,n9xTZEVS8IP4C0arauuyyHyrI_NvR0bz7D-uQV5rJdY9Nc...,Kanriee,7994
20415,EUW1_5162267977,Blue,joR0O9lnLuhRuU8z0OfKJxJU9xxOEguG-HCI_Yarrjm-Fp...,SyRuB1,14513
20416,EUW1_5162267977,Blue,GQIS690djRh5tNELDpJNdxcvyVlwGcqShzRI3krMOeJbjj...,skeith241,6349


In [ ]:
df = pd.read_sql_query("SELECT DISTINCT * from tbl_match_info WHERE summoner_name = 'AdAstraData' AND gold_earned > 0", con=engine)
df['gold_earned'].astype(int).describe()
### AdAstraData

count      112.000000
mean      9048.589286
std       3601.078609
min        550.000000
25%       6694.000000
50%       8803.500000
75%      11406.500000
max      18333.000000
Name: gold_earned, dtype: float64

In [ ]:
df = pd.read_sql_query("SELECT DISTINCT * from tbl_match_info WHERE summoner_name = 'Storm and Sin' AND gold_earned > 0", con=engine)
df['gold_earned'].astype(int).describe()
### Storm and Sin

count      100.000000
mean     11145.490000
std       3752.877472
min        807.000000
25%       9047.500000
50%      11338.000000
75%      13439.500000
max      19373.000000
Name: gold_earned, dtype: float64

In [ ]:
### ACTION_ITEMS : 
# 4. Add new data and test if append respects existing match info

In [ ]:
list_match_item_purchased = []

for var_match_id in last_matches_puuid[0:10]:

    """ store participant's puuids """
    dict_participants_puuids = {}

    match_participants = watcher.match.by_id(region=var_region, match_id=var_match_id)
    list_match_participants = match_participants['metadata']['participants']
    
    for idx in range(len(list_match_participants)):
        participant_id = idx + 1
        dict_participants_puuids[str(participant_id)] = list_match_participants[idx]

    """ fetch match_timeline """
    match_info = watcher.match.timeline_by_match(region=var_region,match_id=var_match_id)
    match_timeline = pd.json_normalize(data=match_info['info']['frames'], record_path='events')

    """ add match_id to match_timeline"""
    match_timeline['match_id'] = var_match_id

    """ convert match_timeline' timestamp to elapsed minutes"""
    match_timeline['timestamp_timedelta'] = pd.to_timedelta(match_timeline['timestamp'], unit='ms')
    match_timeline['elapsed_minutes'] = round(match_timeline['timestamp_timedelta']/ pd.Timedelta(minutes=1),3)
    
    """ subset for item_purchased """
    item_purchased = match_timeline.query("type == 'ITEM_PURCHASED'").dropna(axis=1)

    """ retrieve item_name """
    item_purchased['item_id'] = item_purchased['itemId'].astype(int)
    item_purchased['item_name'] = [dict_item_id_item_name[str(item)] for item in item_purchased['item_id']]

    """ retrieve summoner_puuid """
    item_purchased['participant_id'] = item_purchased['participantId'].astype(int)
    item_purchased['summoner_puuid'] = [dict_participants_puuids[str(id)] for id in item_purchased['participant_id']]

    """ subset for @my_puuid """
    item_purchased = item_purchased.query("summoner_puuid == @my_puuid")

    """ select cols - elapsed_minutes, item_name, summoner_puuid """
    item_purchased = item_purchased[
        [
            'match_id', 
            'elapsed_minutes',
            'item_name',
            'summoner_puuid'
        ]
    ]
    
    """ create dict with item_purchased per timestamp per participant (summoner) """
    dict_item_purchased = item_purchased.to_dict(orient='records')
    list_match_item_purchased.append(dict_item_purchased)

# record end time
end_timer = time.time()

# number of API calls
num_api_calls_per_cycle = 1
num_api_cycles = len(last_matches_puuid)
num_api_calls = num_api_cycles * num_api_calls_per_cycle

print("Execution time:",(end_timer-start_timer), "s")
print("Number of API calls:",num_api_calls)
print("API calls per second:",round(num_api_calls/(end_timer-start_timer),2))

Execution time: 14.254740476608276 s
Number of API calls: 100
API calls per second: 7.02


In [ ]:
""" obtain avg. elapsed_minutes per item_name (last matches)"""
from itertools import chain

df_match_item_purchased = pd.DataFrame(list(chain.from_iterable(list_match_item_purchased)))

avg_min_Dorans_Ring = df_match_item_purchased.query('item_name == "Doran\'s Ring"')['elapsed_minutes'].describe()
avg_min_Dorans_Ring

count    24.000000
mean      0.259708
std       0.146596
min       0.097000
25%       0.155500
50%       0.209000
75%       0.322000
max       0.614000
Name: elapsed_minutes, dtype: float64

In [ ]:
### ACTION_ITEMS : 
# 1. need to test first for API call before proceeding (hitting the API's rate limits as it is) 
# 2. need to add order_num to item_name to distinguish between similar item_names bought at different timestamps
# 3. .describe() to be applied per item_name_order_num 